In [1]:
from random import randrange
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.datasets import load_breast_cancer, load_diabetes


def grad_check_sparse(f, x, analytic_grad, num_checks=10, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Linear regression

In [2]:
data = load_diabetes()
X_train1, y_train1 = data.data, data.target
w1 = np.random.randn(X_train1.shape[1]) * 0.0001
b1 = np.random.randn(1) * 0.0001

In [3]:
def mse_loss_naive(w, b, X, y, alpha=0):
    """
    MSE loss function WITH FOR LOOPs
    
    Returns a tuple of:
    - loss 
    - gradient with respect to weights w
    - gradient with respect to bias b
    """
    
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0.0
    
    # YOUR CODE HERE
    size_y = len(y)
    y_pred = X.dot(w) + b
    
    # loss
    for i in range(size_y):
        loss = loss + (y[i] - y_pred[i])**2
    loss += alpha * w.T.dot(w)
    loss *= (1 / size_y)
    
    # gradient with respect to weights w
    for i in range(size_y):
        dw += X[i] * (y[i] - y_pred[i])
    dw += alpha * w
    dw *= (-2 / size_y)
    
    # gradient with respect to weights b
    for i in range(size_y):
        db += (y[i] - y_pred[i])
    db *= (-2 / size_y)
        
    return loss, dw, np.array(db).reshape(1,)

## Naive Linear regression loss

In [4]:
loss, dw1, db1 = mse_loss_naive(w1, b1, X_train1, y_train1, alpha=0)

sk_loss = mean_squared_error(X_train1 @ w1 + b1, y_train1)
assert rel_error(loss, sk_loss) < 1e-9
print("Loss error : ",rel_error(loss, sk_loss))

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: mse_loss_naive(w1, b1, X_train1, y_train1, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15,  error=1e-5)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: mse_loss_naive(w1, b1, X_train1, y_train1, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15,  error=1e-5)

Loss error :  1.2512613028603116e-16
Gradient check w
numerical: -0.315455 analytic: -0.315455, relative error: 7.106261e-08
numerical: -1.553186 analytic: -1.553187, relative error: 1.920212e-07
numerical: -0.315455 analytic: -0.315455, relative error: 7.106261e-08
numerical: -4.296089 analytic: -4.296087, relative error: 2.120273e-07
numerical: -1.553186 analytic: -1.553187, relative error: 1.920212e-07
numerical: -1.553186 analytic: -1.553187, relative error: 1.920212e-07
numerical: -2.801913 analytic: -2.801913, relative error: 2.570945e-08
numerical: -3.153317 analytic: -3.153317, relative error: 6.301220e-08
numerical: -0.315455 analytic: -0.315455, relative error: 7.106261e-08
numerical: -1.275044 analytic: -1.275043, relative error: 2.286572e-07
numerical: -1.553186 analytic: -1.553187, relative error: 1.920212e-07
numerical: -4.145422 analytic: -4.145424, relative error: 1.948690e-07
numerical: -1.275044 analytic: -1.275043, relative error: 2.286572e-07
numerical: -0.315455 an

## Naive Ridge regression loss

In [5]:
loss, dw1, db1 = mse_loss_naive(w1, b1, X_train1, y_train1, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: mse_loss_naive(w1, b1, X_train1, y_train1, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15,  error=1e-5)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: mse_loss_naive(w1, b1, X_train1, y_train1, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15,  error=1e-5)

Gradient check w
numerical: -1.376396 analytic: -1.376394, relative error: 7.163157e-07
numerical: -4.296089 analytic: -4.296087, relative error: 2.115305e-07
numerical: -3.234126 analytic: -3.234125, relative error: 1.835279e-07
numerical: -1.275044 analytic: -1.275043, relative error: 1.503637e-07
numerical: -4.296089 analytic: -4.296087, relative error: 2.115305e-07
numerical: -1.275044 analytic: -1.275043, relative error: 1.503637e-07
numerical: -2.801912 analytic: -2.801913, relative error: 1.895648e-07
numerical: 2.892060 analytic: 2.892060, relative error: 1.468978e-07
numerical: -3.153318 analytic: -3.153316, relative error: 2.935326e-07
numerical: -1.275044 analytic: -1.275043, relative error: 1.503637e-07
numerical: -2.801912 analytic: -2.801913, relative error: 1.895648e-07
numerical: -0.315456 analytic: -0.315454, relative error: 2.557029e-06
numerical: 2.892060 analytic: 2.892060, relative error: 1.468978e-07
numerical: -1.376396 analytic: -1.376394, relative error: 7.1631

In [6]:
def mse_loss_vectorized(w, b, X, y, alpha=0):
    """
    MSE loss function WITHOUT FOR LOOPs
    
    Returns a tuple of:
    - loss 
    - gradient with respect to weights w
    - gradient with respect to bias b
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0.0
    
    # YOUR CODE HERE
    y_pred = X.dot(w) + b
    
    # loss
    s1 =  (y_pred - y).T @ (y_pred - y) + alpha * (w.T @ w)
    loss =  (1/(y.shape[0])) * s1
    
    # gradient with respect to weights w
    s1 = alpha * w
    s2 = X.T @ (y_pred - y)
    dw = (s1 + s2) * (2/y.shape[0])
    
    # gradient with respect to weights b
    s3 = np.sum(y_pred - y)
    db = s3 * (2/y.shape[0])
    
    return loss, dw, np.array(db).reshape(1,)

## Vectorised Linear regression loss

In [7]:
loss, dw1, db1 = mse_loss_vectorized(w1, b1, X_train1, y_train1, alpha=0)

sk_loss = mean_squared_error(X_train1 @ w1 + b1, y_train1)
assert rel_error(loss, sk_loss) < 1e-9
print("Loss error : ",rel_error(loss, sk_loss))

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: mse_loss_vectorized(w1, b1, X_train1, y_train1, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15,  error=1e-5)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: mse_loss_vectorized(w1, b1, X_train1, y_train1, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15,  error=1e-5)

Loss error :  6.256306514301555e-17
Gradient check w
numerical: -4.296087 analytic: -4.296087, relative error: 3.242401e-10
numerical: -3.153317 analytic: -3.153317, relative error: 2.351522e-08
numerical: -3.234125 analytic: -3.234124, relative error: 4.504793e-08
numerical: -4.296087 analytic: -4.296087, relative error: 3.242401e-10
numerical: 2.892061 analytic: 2.892061, relative error: 7.284076e-09
numerical: -0.315455 analytic: -0.315455, relative error: 7.106261e-08
numerical: -1.275043 analytic: -1.275043, relative error: 8.599622e-08
numerical: -3.234125 analytic: -3.234124, relative error: 4.504793e-08
numerical: 2.892061 analytic: 2.892061, relative error: 7.284076e-09
numerical: -3.153317 analytic: -3.153317, relative error: 2.351522e-08
numerical: -4.296087 analytic: -4.296087, relative error: 3.242401e-10
numerical: -3.234125 analytic: -3.234124, relative error: 4.504793e-08
numerical: -1.275043 analytic: -1.275043, relative error: 8.599622e-08
numerical: 2.892061 analytic

## Vectorized ridge regression loss

In [8]:
loss, dw1, db1 = mse_loss_vectorized(w1, b1, X_train1, y_train1, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: mse_loss_vectorized(w1, b1, X_train1, y_train1, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15,  error=1e-5)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: mse_loss_vectorized(w1, b1, X_train1, y_train1, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15,  error=1e-5)

Gradient check w
numerical: -0.315456 analytic: -0.315456, relative error: 3.926560e-07
numerical: 2.892061 analytic: 2.892061, relative error: 2.722201e-08
numerical: -3.153318 analytic: -3.153318, relative error: 2.329584e-08
numerical: -3.153318 analytic: -3.153318, relative error: 2.329584e-08
numerical: 2.892061 analytic: 2.892061, relative error: 2.722201e-08
numerical: -1.275043 analytic: -1.275043, relative error: 9.295927e-08
numerical: -2.801912 analytic: -2.801912, relative error: 1.376528e-08
numerical: 2.892061 analytic: 2.892061, relative error: 2.722201e-08
numerical: -4.145423 analytic: -4.145423, relative error: 2.159697e-08
numerical: -1.275043 analytic: -1.275043, relative error: 9.295927e-08
numerical: -4.296087 analytic: -4.296087, relative error: 2.199134e-08
numerical: -3.153318 analytic: -3.153318, relative error: 2.329584e-08
numerical: 2.892061 analytic: 2.892061, relative error: 2.722201e-08
numerical: -1.376394 analytic: -1.376394, relative error: 4.457020e-

# Logistic regression

In [9]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

data = load_breast_cancer()
X_train2, y_train2 = data.data, data.target
w2 = np.random.randn(X_train2.shape[1]) * 0.0001
b2 = np.random.randn(1) * 0.0001

# Naive

In [10]:
def logistic(y, m):
    res = y * np.log(m) + (1-y) * np.log(1-m)
    return res

In [11]:
def log_loss_naive(w, b, X, y, alpha=0):
    """
    log loss function WITH FOR LOOPs
    
    Returns a tuple of:
    - loss 
    - gradient with respect to weights w
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0.0
    
    # YOUR CODE HERE
    
    # loss
    for i in range(y.shape[0]):
        sig = sigmoid(w.T.dot(X[i]) + b)
        loss -= logistic(y[i], sig)
    
    loss += alpha * w.T.dot(w)
    loss = (1/y.shape[0]) * loss
    
    # gradient with respect to weights w
    for i in range(w.shape[0]):
        s1 = 2 * alpha * w[i]
        for j in range(y.shape[0]):
            sig = sigmoid((w.T @ X[j]) + b)
            s1 += (sig - y[j]) * X[j][i]
        dw[i] = s1/y.shape[0]
        
    # gradient with respect to bias b
    s2 = 0
    for i in range(y.shape[0]):
        sig = sigmoid((w.T @ X[i]) + b)
        s2 += sig - y[i]
    db = s2 * (1/y.shape[0])
    
    return loss, dw, np.array(db).reshape(1,)

In [12]:
y_pred_0 = sigmoid(X_train2 @ w2 + b2)
y_pred = np.vstack([1-y_pred_0, y_pred_0]).T
sk_loss = log_loss(y_train2, y_pred)

loss, dw2, db2 = log_loss_naive(w2, b2, X_train2, y_train2, alpha=0)
assert rel_error(loss, sk_loss) < 1e-9
print("Loss error : ",rel_error(loss, sk_loss))

print("Gradient check w")
# Check with numerical gradient w
f = lambda w2: log_loss_naive(w2, b2, X_train2, y_train2, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w2, dw2, 15, error=1e-4)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b2: log_loss_naive(w2, b2, X_train2, y_train2, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b2, db2, 15,  error=1e-5)

Loss error :  8.026681113836711e-17
Gradient check w
numerical: -0.000580 analytic: -0.000580, relative error: 3.027572e-09
numerical: -0.000580 analytic: -0.000580, relative error: 3.027572e-09
numerical: -0.010473 analytic: -0.010473, relative error: 2.776260e-09
numerical: 31.900470 analytic: 31.900418, relative error: 8.179354e-07
numerical: 0.009682 analytic: 0.009682, relative error: 1.189431e-09
numerical: 0.021217 analytic: 0.021217, relative error: 1.421578e-09
numerical: -0.008426 analytic: -0.008426, relative error: 2.164990e-09
numerical: 0.009682 analytic: 0.009682, relative error: 1.189431e-09
numerical: 0.014769 analytic: 0.014769, relative error: 1.140001e-09
numerical: 0.007913 analytic: 0.007913, relative error: 3.249854e-09
numerical: -1.724903 analytic: -1.724903, relative error: 1.431440e-10
numerical: -0.008446 analytic: -0.008446, relative error: 1.086927e-09
numerical: -0.019891 analytic: -0.019891, relative error: 1.204096e-09
numerical: 0.021217 analytic: 0.02

# Naive with regulariztion

In [13]:
loss, dw2, db2 = log_loss_naive(w2, b2, X_train2, y_train2, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w2: log_loss_naive(w2, b2, X_train2, y_train2, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w2, dw2, 15, error=1e-4)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b2: log_loss_naive(w2, b2, X_train2, y_train2, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b2, db2, 15,  error=1e-5)

Gradient check w
numerical: -3.660894 analytic: -3.660894, relative error: 9.510353e-09
numerical: -0.008426 analytic: -0.008426, relative error: 2.106727e-09
numerical: 0.010591 analytic: 0.010591, relative error: 1.222721e-09
numerical: 6.557103 analytic: 6.557103, relative error: 3.725920e-09
numerical: -0.164755 analytic: -0.164755, relative error: 1.226403e-10
numerical: 0.014770 analytic: 0.014770, relative error: 1.209110e-09
numerical: 31.900471 analytic: 31.900418, relative error: 8.179353e-07
numerical: -0.008426 analytic: -0.008426, relative error: 2.106727e-09
numerical: -1.724903 analytic: -1.724903, relative error: 1.443631e-10
numerical: 82.072592 analytic: 82.072377, relative error: 1.311562e-06
numerical: 0.155995 analytic: 0.155995, relative error: 1.137257e-11
numerical: -0.000581 analytic: -0.000581, relative error: 3.584415e-09
numerical: -0.164755 analytic: -0.164755, relative error: 1.226403e-10
numerical: 0.155995 analytic: 0.155995, relative error: 1.137257e-11

# Vectorized

In [14]:
def log_loss_vectorized(w, b,X, y, alpha=0):
    """
    log loss function WITHOUT FOR LOOPs
    
    Returns a tuple of:
    - loss 
    - gradient with respect to weights w
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0.0
    
    # YOUR CODE HERE
    # loss
    sig = sigmoid(X.dot(w)+b)
    loss -= sum( y * np.log(sig) + (1-y) * np.log(1-sig) )
    s1 = alpha * w.T.dot(w)
    loss = (loss + s1) / y.shape[0]
    
    # gradient with respect to weights w
    s1 = 2 * alpha * w
    s2 = X.T.dot(sig - y)
    dw = (s1 + s2)/y.shape[0]
    
    # gradient with respect to bias b
    s3 = sum(sig - y)
    db = s3 / y.shape[0]
    
    return loss, dw, np.array(db).reshape(1,)

In [15]:
y_pred_0 = sigmoid(X_train2 @ w2 + b2)
y_pred = np.vstack([1-y_pred_0, y_pred_0]).T
sk_loss = log_loss(y_train2, y_pred)

loss, dw2, db2 = log_loss_vectorized(w2, b2, X_train2, y_train2, alpha=0)
assert rel_error(loss, sk_loss) < 1e-9
print("Loss error : ",rel_error(loss, sk_loss))

print("Gradient check w")
# Check with numerical gradient w
f = lambda w2: log_loss_vectorized(w2, b2, X_train2, y_train2, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w2, dw2, 15, error=1e-4)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b2: log_loss_vectorized(w2, b2, X_train2, y_train2, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b2, db2, 15,  error=1e-5)

Loss error :  1.6053362227673423e-16
Gradient check w
numerical: 0.010591 analytic: 0.010591, relative error: 1.215677e-09
numerical: 0.014769 analytic: 0.014769, relative error: 1.140001e-09
numerical: -1.724903 analytic: -1.724903, relative error: 1.431438e-10
numerical: -2.090239 analytic: -2.090239, relative error: 3.075403e-10
numerical: -0.008426 analytic: -0.008426, relative error: 2.164990e-09
numerical: 0.001163 analytic: 0.001163, relative error: 2.344502e-08
numerical: -0.000408 analytic: -0.000408, relative error: 1.452814e-08
numerical: 0.009682 analytic: 0.009682, relative error: 1.189430e-09
numerical: 0.009682 analytic: 0.009682, relative error: 1.189430e-09
numerical: -0.002769 analytic: -0.002769, relative error: 5.994823e-09
numerical: -0.164755 analytic: -0.164755, relative error: 1.145457e-10
numerical: -0.000884 analytic: -0.000884, relative error: 9.012045e-09
numerical: -0.000367 analytic: -0.000367, relative error: 3.534357e-09
numerical: 0.007913 analytic: 0.0

# Vectorized with regularization

In [16]:
loss, dw2, db2 = log_loss_vectorized(w2, b2, X_train2, y_train2, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w2: log_loss_vectorized(w2, b2, X_train2, y_train2, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w2, dw2, 15, error=1e-4)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b2: log_loss_vectorized(w2, b2, X_train2, y_train2, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b2, db2, 15,  error=1e-5)

Gradient check w
numerical: -0.001037 analytic: -0.001037, relative error: 4.610958e-10
numerical: 0.007913 analytic: 0.007913, relative error: 2.886004e-09
numerical: -0.010472 analytic: -0.010472, relative error: 2.707419e-09
numerical: -2.090240 analytic: -2.090240, relative error: 3.074138e-10
numerical: 31.900471 analytic: 31.900418, relative error: 8.179352e-07
numerical: 31.900471 analytic: 31.900418, relative error: 8.179352e-07
numerical: -0.008446 analytic: -0.008446, relative error: 8.618521e-10
numerical: 0.007913 analytic: 0.007913, relative error: 2.886004e-09
numerical: 31.900471 analytic: 31.900418, relative error: 8.179352e-07
numerical: 0.001163 analytic: 0.001163, relative error: 2.278968e-08
numerical: -0.000407 analytic: -0.000407, relative error: 1.142815e-08
numerical: -0.000368 analytic: -0.000368, relative error: 5.746886e-10
numerical: 0.014770 analytic: 0.014770, relative error: 1.209109e-09
numerical: -3.660894 analytic: -3.660894, relative error: 9.509596e-

# Gradient descent for Linear models

In [17]:
class LinearModel():
    def __init__(self):
        self.w = None
        self.b = None

    def train(self, X, y, learning_rate=1e-3, alpha=0, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            rand = np.random.choice(N, batch_size, replace=False)
            
            X_batch = X[rand, :]
            y_batch = y[rand]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update                                                                
            # Update the weights w and bias b using the gradient and the learning rate.          
            self.w -= learning_rate * dw
            self.b -= learning_rate * db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
                
        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class LinearRegressor(LinearModel):
    """ Linear regression """

    def loss(self, X_batch, y_batch, alpha):
        return mse_loss_vectorized(self.w, self.b, X_batch, y_batch, alpha)
    
    def predict(self, X):
        y = X.dot(self.w) + self.b
        return y

class LogisticRegressor(LinearModel):
    """ Linear regression """

    def loss(self, X_batch, y_batch, alpha):
        return log_loss_vectorized(self.w, self.b, X_batch, y_batch, alpha)
    
    def predict(self, X):
        """ Return prediction labels vector of 0 or 1 """
        # YOUR CODE HERE
        y = X.dot(self.w) + self.b
        r = np.zeros_like(y)
        for i in range(len(y)):
            r[i] = 1 if y[i]>0 else 0
        return r

## Linear regression with gradient descent

In [18]:
from sklearn.linear_model import LinearRegression

sk_model = LinearRegression(fit_intercept=True)
sk_model.fit(X_train1, y_train1)
sk_pred = sk_model.predict(X_train1)
sk_mse = mean_squared_error(sk_pred, y_train1)

model = LinearRegressor()
model.train(X_train1, y_train1, num_iters=75000, batch_size=64, learning_rate=1e-2, verbose=True)
pred = model.predict(X_train1)
mse = mean_squared_error(pred, y_train1)

print("MSE scikit-learn:", sk_mse)
print("MSE gradient descent model :", mse)
assert mse - sk_mse < 100

iteration 0 / 75000: loss 31775.487941
iteration 10000 / 75000: loss 2951.152194
iteration 20000 / 75000: loss 3723.112860
iteration 30000 / 75000: loss 2426.692002
iteration 40000 / 75000: loss 3985.396419
iteration 50000 / 75000: loss 2580.550531
iteration 60000 / 75000: loss 2815.209782
iteration 70000 / 75000: loss 2529.564721
MSE scikit-learn: 2859.6903987680657
MSE gradient descent model : 2884.97938561255


## Logistc regression with gradient descent

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train2)

sk_model = LogisticRegression(fit_intercept=True)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_log_loss = log_loss(sk_pred, y_train2)

model = LogisticRegressor()
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_log_loss = log_loss(pred, y_train2)

print("Log-loss scikit-learn:", sk_log_loss)
print("Log-loss gradiet descent model :", model_log_loss)
print("Error :", rel_error(sk_log_loss, model_log_loss))
assert rel_error(sk_log_loss, model_log_loss) < 1e-7

iteration 0 / 75000: loss 0.693404
iteration 10000 / 75000: loss 0.087601
iteration 20000 / 75000: loss 0.068377
iteration 30000 / 75000: loss 0.127894
iteration 40000 / 75000: loss 0.051696
iteration 50000 / 75000: loss 0.065748
iteration 60000 / 75000: loss 0.045060
iteration 70000 / 75000: loss 0.045071
Log-loss scikit-learn: 0.4249086712816093
Log-loss gradiet descent model : 0.4249086712816093
Error : 0.0
